# Building a YOLO Object Detector with Darknet in the Cloud!

#### **This notebook was copied and adapted from https://github.com/theAIGuysCode/YOLOv3-Cloud-Tutorial**, which includes a step-by-step explanation at https://www.youtube.com/watch?v=10joRJt39Ns.

# Step 1: Configure Darknet, YOLO, mount Google Drive

### How to run this code
1. Copy all required files to a folder in your drive:
  - empty folder named 'backup', where training weights will be stored
  - generate_train.py
  - generate_test.py
  - obj.data
  - obj.names
  - obj.zip
  - val.zip
  - cfg file
2. In the first line of code, define the path to your folder.
3. Run the code.


### The input labelling data for training YOLO is the in following format:
    class x y width height

- with *class* being the number of the class (e.g., 0, 1, 2, ...)

- with *x* and *y* being the center xy coordinate of the bounding box, normalized from 0 to 1 relative to the image size

- with *width* and *height* being the width and height of the bounding box, normalized from 0 to 1 relative to the image size


### The output bounding box data is in the following format:
    class x y width height

- with *class* being the name of the class and its probability

- with *x* and *y* being the top left xy coordinate of the bounding box, in pixels

- with *width* and *height* being the width and height of the bounding box, in pixels



In [ ]:
# Define the path to the folder in your drive
foldername = "foldername"

In [ ]:
# clone darknet repo
!git clone https://github.com/AlexeyAB/darknet

In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!echo "ARCH= -gencode arch=compute_75,code=sm_75" >> Makefile

In [ ]:
# verify CUDA
!/usr/local/cuda/bin/nvcc --version

In [ ]:
# make darknet (build)
!make

In [ ]:
# define helper functions
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

# use this to upload files
def upload():
  from google.colab import files
  uploaded = files.upload()
  for name, data in uploaded.items():
    with open(name, 'wb') as f:
      f.write(data)
      print ('saved file', name)

# use this to download a file
def download(path):
  from google.colab import files
  files.download(path)

In [ ]:
%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# this creates a symbolic link so that now the path /content/gdrive/My\ Drive/ is equal to /mydrive
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

# Step 2: Moving Your Custom Dataset Into Your Cloud VM

In [ ]:
# this is where my zip is stored (I created a yolov3 folder where I will get my required files from)
!ls /mydrive/{foldername}

In [ ]:
# copy the .zip file into the root directory of cloud VM
!cp /mydrive/{foldername}/obj.zip ../

In [ ]:
# unzip the zip file and its contents should now be in /data/obj
!unzip ../obj.zip -d data/

# Step 3: Configuring Files for Training

Additional information: https://www.youtube.com/watch?v=zJDUhGL26iU&t=300s

In [ ]:
# upload the custom .cfg back to cloud VM from Google Drive
!cp /mydrive/{foldername}/yolov7-tiny.cfg ./yolov7-tiny.cfg

# upload the custom .cfg back to cloud VM from local machine (uncomment to use)
#%cd cfg
#upload()
#%cd ..

## ii) obj.names and obj.data



In [ ]:
# upload the obj.names and obj.data files to cloud VM from Google Drive
!cp /mydrive/{foldername}/obj.names ./data
!cp /mydrive/{foldername}/obj.data  ./data

# upload the obj.names and obj.data files to cloud VM from local machine (uncomment to use)
#%cd data
#upload()
#%cd ..

## iii) Generating train.txt


In [ ]:
# upload the generate_train.py script to cloud VM from Google Drive
!cp /mydrive/{foldername}/generate_train.py ./

# upload the generate_train.py script to cloud VM from local machine (uncomment to use)
#upload()


Now we simply run the python script to do all the work for us.

In [ ]:
!python generate_train.py

In [ ]:
# verify train.txt can be seen in our /data folder
!ls data/

# Step 4: Download pre-trained weights for the convolutional layers.


In [ ]:
# upload pretrained convolutional layer weights
#!wget http://pjreddie.com/media/files/darknet53.conv.74

#!wget https://pjreddie.com/media/files/yolov3-tiny.weights

In [ ]:
# This stops 'Run all' at this cell by causing an error
assert False

# Step 5: Train Your Custom Object Detector!


In [ ]:
# train your custom detector
!darknet/darknet detector train data/obj.data ./yolov7-tiny.cfg -dont_show -clear

You can observe a chart of how your model did throughout the training process by running the below command. It shows a chart of your average loss vs. iterations. For your model to be 'accurate' you would aim for a loss under 2.

In [ ]:
imShow('chart.png')

In [ ]:
# continue training using the last-saved weights (if training was interrupted)
!darknet/darknet detector train data/obj.data ./yolov7-tiny.cfg /mydrive/{foldername}/backup/yolov7-tiny_last.weights -dont_show

# Step 6: Validate Your Custom Object Detector

In [ ]:
# copy the .zip file into the root directory of cloud VM
!cp /mydrive/{foldername}/val.zip ../

In [ ]:
# unzip the zip file and its contents should now be in /data/val
!unzip ../val.zip -d data/

In [ ]:
# upload the generate_test.py script to cloud VM from Google Drive
!cp /mydrive/{foldername}/generate_test.py ./

In [ ]:
!python generate_test.py

In [ ]:
# verify test.txt can be seen in our /data folder
!ls data/

In [ ]:
# To evaluate the model’s performance over the test set, run the following command:
!darknet/darknet detector map data/obj.data ./yolov7-tiny.cfg /mydrive/{foldername}/backup/yolov7-tiny_last.weights -thresh 0.8 -iou_thresh 0.7

# Step 7: Run Your Custom Object Detector

In [ ]:
# need to set our custom cfg to test mode
!sed -i 's/batch=64/batch=1/' yolov7-tiny.cfg
!sed -i 's/subdivisions=2/subdivisions=1/' yolov7-tiny.cfg



In [ ]:
# run your custom detector with this command (upload an image to your google drive to test (e.g., named img.jpg))
!darknet/darknet detector test data/obj.data ./yolov7-tiny.cfg /mydrive/{foldername}/backup/yolov7-tiny_final.weights /mydrive/{foldername}/img.jpg -thresh 0.8  -ext_output > output.txt
imShow('predictions.jpg')

In [ ]:
# download results to google drive
!cp predictions.jpg /mydrive/{foldername}/img_prediction.jpg
!cp output.txt /mydrive/{foldername}/img_prediction.txt